# Phase 3 — Risk Scoring and Memory

Apply temporal smoothing, hysteresis, and oscillation detection to perception signals.

## Imports and setup

In [ ]:
from pathlib import Path
import sys

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils import load_config, ensure_demo_video, open_capture, iter_frames, release_capture
from src.perception_flow import compute_optical_flow, compute_obstacle_risk, compute_openness
from src.memory import NavigationMemory

config = load_config("configs/default.yaml")
video_path = ensure_demo_video(config)
print(f"Video: {video_path}")

## Process video with memory

Apply perception and memory to full video sequence.

In [ ]:
cap = open_capture(config)
max_frames = config["video"].get("max_frames") or 200

# Initialize memory
memory = NavigationMemory(config)

# Storage for plotting
results = {
    "risk_raw": [],
    "risk_smooth": [],
    "openness_raw": [],
    "openness_smooth": [],
    "avoid_state": [],
    "stop_state": [],
    "oscillating": [],
    "in_cooldown": [],
    "preferred_direction": [],
}

prev_gray = None

for idx, frame in tqdm(iter_frames(cap, max_frames=max_frames, convert_rgb=False), total=max_frames):
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev_gray is not None:
        # Compute perception
        flow = compute_optical_flow(prev_gray, curr_gray, config)
        risk_raw = compute_obstacle_risk(flow, config)
        openness_raw, direction = compute_openness(flow, config)
        
        # Update memory
        signals = memory.update(risk_raw, openness_raw, direction)
        
        # Store results
        for key in results:
            if key == "preferred_direction":
                results[key].append(1 if signals[key] == "right" else -1)
            else:
                results[key].append(signals[key])
    
    prev_gray = curr_gray

release_capture(cap)

print(f"Processed {len(results['risk_raw'])} frames")
print(f"Avoid state triggered: {sum(results['avoid_state'])} times")
print(f"Stop state triggered: {sum(results['stop_state'])} times")
print(f"Oscillation detected: {sum(results['oscillating'])} times")

## Visualize smoothing effect

In [ ]:
frames = np.arange(len(results["risk_raw"]))

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Risk: raw vs smoothed
axes[0].plot(frames, results["risk_raw"], label="Raw Risk", alpha=0.4, color="lightcoral")
axes[0].plot(frames, results["risk_smooth"], label="Smoothed Risk", color="red", linewidth=2)
axes[0].axhline(config["risk"]["thresholds"]["avoid"], color="orange", linestyle="--", label="Avoid Threshold")
axes[0].axhline(config["risk"]["thresholds"]["stop"], color="darkred", linestyle="--", label="Stop Threshold")
axes[0].set_ylabel("Risk Score")
axes[0].set_ylim([0, 1])
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_title("Temporal Smoothing Effect")

# Openness: raw vs smoothed
axes[1].plot(frames, results["openness_raw"], label="Raw Openness", alpha=0.4, color="lightblue")
axes[1].plot(frames, results["openness_smooth"], label="Smoothed Openness", color="blue", linewidth=2)
axes[1].axhline(0, color="black", linestyle="-", linewidth=0.5)
axes[1].set_ylabel("Openness (L-R)")
axes[1].set_xlabel("Frame")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Visualize hysteresis and state transitions

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Risk with hysteresis states
axes[0].plot(frames, results["risk_smooth"], label="Smoothed Risk", color="red", linewidth=2)
axes[0].fill_between(frames, 0, 1, where=results["avoid_state"], alpha=0.2, color="orange", label="Avoid State")
axes[0].fill_between(frames, 0, 1, where=results["stop_state"], alpha=0.3, color="darkred", label="Stop State")
axes[0].axhline(config["risk"]["thresholds"]["avoid"], color="orange", linestyle="--", alpha=0.5)
axes[0].axhline(config["risk"]["hysteresis"]["leave_avoid"], color="orange", linestyle=":", alpha=0.5)
axes[0].axhline(config["risk"]["thresholds"]["stop"], color="darkred", linestyle="--", alpha=0.5)
axes[0].axhline(config["risk"]["hysteresis"]["leave_stop"], color="darkred", linestyle=":", alpha=0.5)
axes[0].set_ylabel("Risk Score")
axes[0].set_ylim([0, 1])
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_title("Hysteresis Thresholding")

# Preferred direction with oscillation detection
axes[1].plot(frames, results["preferred_direction"], label="Preferred Direction", color="green", drawstyle="steps-post")
axes[1].fill_between(frames, -1.5, 1.5, where=results["oscillating"], alpha=0.3, color="red", label="Oscillating")
axes[1].axhline(0, color="black", linestyle="-", linewidth=0.5)
axes[1].set_ylabel("Direction (±1)")
axes[1].set_ylim([-1.5, 1.5])
axes[1].set_yticks([-1, 0, 1])
axes[1].set_yticklabels(["Left", "Center", "Right"])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Cooldown periods
axes[2].fill_between(frames, 0, 1, where=results["in_cooldown"], alpha=0.4, color="purple", label="In Cooldown")
axes[2].set_ylabel("Cooldown")
axes[2].set_xlabel("Frame")
axes[2].set_ylim([0, 1])
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## State transition analysis

In [ ]:
# Count state transitions
avoid_transitions = 0
stop_transitions = 0
direction_flips = 0

for i in range(1, len(results["avoid_state"])):
    if results["avoid_state"][i] != results["avoid_state"][i-1]:
        avoid_transitions += 1
    if results["stop_state"][i] != results["stop_state"][i-1]:
        stop_transitions += 1
    if results["preferred_direction"][i] != results["preferred_direction"][i-1]:
        direction_flips += 1

print("=" * 50)
print("State Transition Summary")
print("=" * 50)
print(f"Total frames processed: {len(results['risk_raw'])}")
print(f"Avoid state transitions: {avoid_transitions}")
print(f"Stop state transitions: {stop_transitions}")
print(f"Direction flips: {direction_flips}")
print(f"Oscillation events: {sum(results['oscillating'])}")
print(f"Cooldown activations: {sum(np.diff([0] + results['in_cooldown']) > 0)}")
print("\nMemory system successfully reduces jitter and prevents rapid flip-flopping.")

## Verification

Phase 3 verification:
- Smoothed signals show reduced jitter compared to raw signals
- Hysteresis prevents rapid state transitions
- Oscillation detection identifies flip-flopping behavior
- Cooldown mechanism prevents excessive direction changes